## Keolis Inventory Report Script
##### Created by Bridget Leary

In [1]:
# Import statements
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta, MO
from datetime import datetime, timedelta
import copy

#### Excel Data Preparation

In [2]:
# Import Unfiltered Roll Forward Report from Will
# If you want to create a report for a month that is not the month prior to now, change these variables

report_date = datetime.now() - pd.offsets.MonthEnd(n=1)
year_month = str(report_date.year) + '-' + str('%02d' % report_date.month) + ' ' + str(report_date.strftime('%b')) # format: '2023-02 Feb'
month_name = str(report_date.strftime('%B')) # format: 'February'

path = 'O:\\PP-PROLOG\\VENDOR_MANAGEMENT\\Data_Analytics\\Data_Files\\Inventory_Mgmt_Data\\7_Keolis_Inventory\\Keolis Reports\\Keolis Roll Forward Reports\\Unfiltered Roll Forward Report - ' + year_month + '.xlsx'

FinalMonth = pd.read_excel(path, sheet_name = "Final " + month_name, dtype = "string")

TxnHistory = pd.read_excel(path, sheet_name = "MBTA Transaction History KCS", dtype = "string")

In [3]:
# Delete Totals Rows
FinalMonth.drop(FinalMonth.tail(3).index,inplace=True)
TxnHistory.drop(TxnHistory.tail(3).index,inplace=True)

In [4]:
# Import Template sheets
TempIOH = pd.read_excel("O:\\PP-PROLOG\\VENDOR_MANAGEMENT\\Data_Analytics\\Data_Files\\Inventory_Mgmt_Data\\7_Keolis_Inventory\\Keolis Reports\\MPT Ending Inventory Template.xlsx", sheet_name = "Import IOH to Access", dtype = "string")

TempTXN = pd.read_excel("O:\\PP-PROLOG\\VENDOR_MANAGEMENT\\Data_Analytics\\Data_Files\\Inventory_Mgmt_Data\\7_Keolis_Inventory\\Keolis Reports\\MPT Ending Inventory Template.xlsx", sheet_name = "Import Txns to Access", dtype = "string")

In [5]:
# Copy over IOH to template
report_date = datetime.now() - pd.offsets.MonthEnd(n=1)

FinalMonth.insert(0, 'Report Date', report_date)

FinalMonth['Report Date'] = FinalMonth['Report Date'].astype("string")
FinalMonth.head(5)

,Report Date,Part Suffix Location,Part Suffix,Part ID,Suffix,Location,Location Description,Part Description,MPT Status,Final QOH,Final Unit Cost,Final Extended Cost
0,2023-06-30 09:48:36.910683,0271401749-0-406,0271401749-0,0271401749,0,406,COBBLE HILL SIGNAL STORES,ELECTRIC SPACE HEATER TPI CORP. #188-TASA AND...,NON-MPT,20,-7.562,-151.24
1,2023-06-30 09:48:36.910683,2471400558-1-400,2471400558-1,2471400558,1,400,BET - STORES,9-ASPECT CAB SIGNAL; ENCLOSURE W/CARDFILE; F/R...,MPT,1,0,0
2,2023-06-30 09:48:36.910683,2600COMBO1-0-400SP,2600COMBO1-0,2600COMBO1,0,400SP,SERIALIZED PARTS-COMP STAGING,LOCOMOTIVE TRACTION MOTOR WHEEL SET COMBO,MPT,25,0,0
3,2023-06-30 09:48:36.910683,2651985084-1-400SP,2651985084-1,2651985084,1,400SP,SERIALIZED PARTS-COMP STAGING,MOTOR TRACTION D-77 20 TOOTH,MPT,369,0,0
4,2023-06-30 09:48:36.910683,2670300488-0-400,2670300488-0,2670300488,0,400,BET - STORES,CARD FILE ASSY F/ACSES BOARDS----COMPLETE,MPT,1,0,0


In [6]:
# Add data to template for Inventory on Hand
TempIOH_Header = TempIOH[0:0] # keep header from template
df_IOH = pd.concat([TempIOH_Header, FinalMonth])

df_IOH

,Report Date,Part Suffix Location,Part Suffix,Part ID,Suffix,Location,Part Description,MPT Status,Final QOH,Final Unit Cost,Final Extended Cost,Initial QOH,Initial Unit Price,Initial Value OH,Initial MPT Status,Location Description
0,2023-06-30 09:48:36.910683,0271401749-0-406,0271401749-0,0271401749,0,406,ELECTRIC SPACE HEATER TPI CORP. #188-TASA AND...,NON-MPT,20,-7.562,-151.24,<NA>,<NA>,<NA>,<NA>,COBBLE HILL SIGNAL STORES
1,2023-06-30 09:48:36.910683,2471400558-1-400,2471400558-1,2471400558,1,400,9-ASPECT CAB SIGNAL; ENCLOSURE W/CARDFILE; F/R...,MPT,1,0,0,<NA>,<NA>,<NA>,<NA>,BET - STORES
2,2023-06-30 09:48:36.910683,2600COMBO1-0-400SP,2600COMBO1-0,2600COMBO1,0,400SP,LOCOMOTIVE TRACTION MOTOR WHEEL SET COMBO,MPT,25,0,0,<NA>,<NA>,<NA>,<NA>,SERIALIZED PARTS-COMP STAGING
3,2023-06-30 09:48:36.910683,2651985084-1-400SP,2651985084-1,2651985084,1,400SP,MOTOR TRACTION D-77 20 TOOTH,MPT,369,0,0,<NA>,<NA>,<NA>,<NA>,SERIALIZED PARTS-COMP STAGING
4,2023-06-30 09:48:36.910683,2670300488-0-400,2670300488-0,2670300488,0,400,CARD FILE ASSY F/ACSES BOARDS----COMPLETE,MPT,1,0,0,<NA>,<NA>,<NA>,<NA>,BET - STORES


In [7]:
# Copy over Txns to template

# Matching the columns
TxnHistory = TxnHistory[['Part ID', 'Suffix', 'Part Suffix', 'Part Description', 
                        'Funding Status', 'Part Classification', 'Part Classification Description',
                        'Product Category', 'Product Categroy Description', 'TXN Date', 'TXN Type',
                        'Location', 'TXN QTY', 'Before Unit Price', 'After Unit Price',  'UOM',
                        'TNX Value', 'GL Account Code','Unique Id', 'MPT Status', 'PO Number']] # only extracting columns in the template

TxnHistory.head(5)

,Part ID,Suffix,Part Suffix,Part Description,Funding Status,Part Classification,Part Classification Description,Product Category,Product Categroy Description,TXN Date,...,Location,TXN QTY,Before Unit Price,After Unit Price,UOM,TNX Value,GL Account Code,Unique Id,MPT Status,PO Number
0,0170301332,0,0170301332-0,BIT; TIE BORER; 11/16`` / 2-FLUTE,OF,C1,PURCHASE NEW ONLY,01,TRACK DEPARTMENT MATERIAL RAIL,2023-01-13 00:00:00,...,405,0,0,0,EACH,0,<NA>,<NA>,NON-MPT,P000001781
1,0199906319,0,0199906319-0,ANCHOR- RAIL- UNIT 5 DRIVE ON HEAVY DUTY F/115 RE,OF,C1,PURCHASE NEW ONLY,01,TRACK DEPARTMENT MATERIAL RAIL,2023-03-07 00:00:00,...,405,0,2.04,2.04,EACH,0,<NA>,<NA>,NON-MPT,P000004023
2,0255500188,0,0255500188-0,BRUSH F/ MODEL 3593 GATE MECH OLD STYLE,OF,C1,PURCHASE NEW ONLY,02,SIGNAL EQUIPMENT,2023-04-18 00:00:00,...,406,0,26.25,26.25,EACH,0,<NA>,<NA>,NON-MPT,<NA>
3,0270300361,0,0270300361-0,BELL; ELECTRONIC; F/MOUNTING ON 4`` & 5`` MASTS,OF,C1,PURCHASE NEW ONLY,02,SIGNAL EQUIPMENT,2023-05-10 00:00:00,...,406,0,0,0,EACH,0,<NA>,<NA>,NON-MPT,P000004260
4,0270301664,0,0270301664-0,HANDSET; F/ LOCO RADIO; BLACK TELEPHONE PRIMUS...,OF,C1,PURCHASE NEW ONLY,02,SIGNAL EQUIPMENT,2023-06-06 00:00:00,...,400,0,86.71,86.71,EACH,0,<NA>,<NA>,NON-MPT,<NA>


In [9]:
# Add data to template for Transactions
TempTXN_Header = TempTXN[0:0] # keep header from template
df_TXN = pd.concat([TempTXN_Header, TxnHistory])

df_TXN

,Part ID,Suffix,Part Suffix,Part Description,Funding Status,Part Classification,Part Classification Description,Product Category,Product Categroy Description,TXN Date,...,Location,TXN QTY,Before Unit Price,After Unit Price,UOM,TNX Value,GL Account Code,Unique Id,MPT Status,PO Number
0,0170301332,0,0170301332-0,BIT; TIE BORER; 11/16`` / 2-FLUTE,OF,C1,PURCHASE NEW ONLY,01,TRACK DEPARTMENT MATERIAL RAIL,2023-01-13 00:00:00,...,405,0,0,0,EACH,0,<NA>,<NA>,NON-MPT,P000001781
1,0199906319,0,0199906319-0,ANCHOR- RAIL- UNIT 5 DRIVE ON HEAVY DUTY F/115 RE,OF,C1,PURCHASE NEW ONLY,01,TRACK DEPARTMENT MATERIAL RAIL,2023-03-07 00:00:00,...,405,0,2.04,2.04,EACH,0,<NA>,<NA>,NON-MPT,P000004023
2,0255500188,0,0255500188-0,BRUSH F/ MODEL 3593 GATE MECH OLD STYLE,OF,C1,PURCHASE NEW ONLY,02,SIGNAL EQUIPMENT,2023-04-18 00:00:00,...,406,0,26.25,26.25,EACH,0,<NA>,<NA>,NON-MPT,<NA>
3,0270300361,0,0270300361-0,BELL; ELECTRONIC; F/MOUNTING ON 4`` & 5`` MASTS,OF,C1,PURCHASE NEW ONLY,02,SIGNAL EQUIPMENT,2023-05-10 00:00:00,...,406,0,0,0,EACH,0,<NA>,<NA>,NON-MPT,P000004260
4,0270301664,0,0270301664-0,HANDSET; F/ LOCO RADIO; BLACK TELEPHONE PRIMUS...,OF,C1,PURCHASE NEW ONLY,02,SIGNAL EQUIPMENT,2023-06-06 00:00:00,...,400,0,86.71,86.71,EACH,0,<NA>,<NA>,NON-MPT,<NA>


#### Recreating Microsoft Access Queries

In [11]:
# Paths to Historical Data (output from last month)

All_Txns_Path = "O:\\PP-PROLOG\VENDOR_MANAGEMENT\\Vendor Management Team\\Co-Ops\\BridgetLeary\\Keolis\\New Keolis\\All_Txns.xlsx"
Initial_Inv_Path = "O:\\PP-PROLOG\VENDOR_MANAGEMENT\\Vendor Management Team\\Co-Ops\\BridgetLeary\\Keolis\\New Keolis\\Initial_Inv.xlsx"
Initial_Final_Path = "O:\\PP-PROLOG\VENDOR_MANAGEMENT\\Vendor Management Team\\Co-Ops\\BridgetLeary\\Keolis\\New Keolis\\Initial_Final.xlsx" 

In [12]:
# Read in tables of historical data
All_Txns = pd.read_excel(All_Txns_Path)
Initial_Inv = pd.read_excel(Initial_Inv_Path)
Initial_Final = pd.read_excel(Initial_Final_Path)

In [13]:
# Append new data to historical tables
All_Txns_Updated = pd.concat([All_Txns, df_TXN], ignore_index=True)
Initial_Inv_Updated = pd.concat([Initial_Inv, df_IOH], ignore_index=True)

In [14]:
Initial_Inv_Updated['Report Date'] = pd.to_datetime(Initial_Inv_Updated['Report Date'])
Initial_Inv_Updated['Report Date'] = Initial_Inv_Updated['Report Date'].dt.strftime('%Y-%m-%d') # format '2023-05-31'

In [15]:
# 1. Prior Month Tbl Query

# extracting the necessary columns
Prior_Month_Tbl = Initial_Inv_Updated[['Report Date', 'Location', 'Part Suffix', 'Part Description',
                                        'Final QOH', 'Final Unit Cost', 'Final Extended Cost', 'MPT Status']]

# renaming columns to match query
Prior_Month_Tbl = Prior_Month_Tbl.rename(columns={'Final QOH': 'Initial QOH', 'Final Unit Cost': 'Initial Unit Price',
                                                    'Final Extended Cost': 'Initial VOH', 'MPT Status': 'Initial MPT Status'})
                                                    
# adding calculated field
Prior_Month_Tbl['Report Date'] = pd.to_datetime(Prior_Month_Tbl['Report Date'])

Prior_Month_Tbl['Future Report Date'] = Prior_Month_Tbl['Report Date'] + pd.DateOffset(months=1) + pd.offsets.MonthEnd(0) # one month ahead of report month

Prior_Month_Tbl

,Report Date,Location,Part Suffix,Part Description,Initial QOH,Initial Unit Price,Initial VOH,Initial MPT Status,Future Report Date
0,2021-11-30,400,1087474843-0,CONTACTOR 115 VAC 27 AMP 3 POLE,21.0,NaN,2957.43,MPT,2021-12-31
1,2021-11-30,400,1152256413-0,"RIVET COUNTERSUNK HD 3/16"" X 11/16; EMD 273621",40.0,NaN,28.0,MPT,2021-12-31
2,2021-11-30,400,1171402395-0,SCREW; SCD; FLATHEAD; UNC-2A; F/KNOB(267140491...,6.0,NaN,8.82,MPT,2021-12-31
3,2021-11-30,400,1241404022-0,SPRING COIL OUTER PER AAR STANDARD 12452A,19.0,NaN,1641.04,MPT,2021-12-31
4,2021-11-30,400,1241404023-0,SPRING COIL INNER PER AAR STANDARD,66.0,NaN,4822.08,MPT,2021-12-31
...,...,...,...,...,...,...,...,...,...
417983,2023-06-30,400SW,2471405113-0,"IVNR CONTROLLER INSTALL KIT, GPS-WIFI, MBTA/PU...",32,17662,565184,MPT,2023-07-31
417984,2023-06-30,405,0104500149-0,CLIP- RAIL; (25/BAG); PANDROL E-2055,320905,1.994,639884.57,NON-MPT,2023-07-31
417985,2023-06-30,405,0124014886-0,"PLATE- TIE- PANDROL- 6"" BASE FOR LAG SCREWS",41986,18.4228,773499.67,NON-MPT,2023-07-31
417986,2023-06-30,405,0599900207-0,TIE- CROSS- TREATED,15606,67.1969,1048674.82,NON-MPT,2023-07-31


In [16]:
# 2. Prior Month Carry Forward

# joined table
Initial_Inv_Copy = copy.copy(Initial_Inv_Updated)

Initial_Inv_Copy = Initial_Inv_Copy.rename(columns={'Report Date': 'Future Report Date'})

Initial_Inv_Copy['Future Report Date'] = pd.to_datetime(Initial_Inv_Copy['Future Report Date'])

# join prior month with this month
Join_2 = Prior_Month_Tbl.merge(Initial_Inv_Copy, indicator='i', how='outer', on=['Location', 'Part Suffix', 'Future Report Date']).query('i == "left_only"').drop('i', axis=1)

Join_2['Final QOH'] = np.NaN
Join_2['Final Unit Price'] = np.NaN
Join_2['Final VOH'] = np.NaN
Join_2['MPT Status'] = Join_2['Initial MPT Status_x']

# extracting the necessary columns
Prior_Month_Carry_Forward = Join_2[['Future Report Date', 'Location', 'Part Suffix',
                                                'Part Description_x', 'Initial QOH_x', 'Initial Unit Price_x',
                                                'Initial VOH', 'Initial MPT Status_x', 'Final QOH', 'Final Unit Price', 'Final VOH',
                                                'MPT Status', 'Last Issue Date']]

# renaming columns to keep only the left table's columns
Prior_Month_Carry_Forward = Prior_Month_Carry_Forward.rename(columns={'Part Suffix': 'Item Number-Sfx',
                                                                        'Part Description_x': 'Item Desc',
                                                                        'Initial QOH_x': 'Initial QOH', 'Initial Unit Price_x': 'Initial Unit Price',
                                                                        'Initial MPT Status_x': 'Initial MPT Status'})


Prior_Month_Carry_Forward.reset_index().drop('index', axis=1)

Query_2_Unsorted = Prior_Month_Carry_Forward

Prior_Month_Carry_Forward = Prior_Month_Carry_Forward.sort_values(['Future Report Date', 'Item Number-Sfx'], ascending = [True, True], ignore_index=True)

Prior_Month_Carry_Forward

,Future Report Date,Location,Item Number-Sfx,Item Desc,Initial QOH,Initial Unit Price,Initial VOH,Initial MPT Status,Final QOH,Final Unit Price,Final VOH,MPT Status,Last Issue Date
0,2021-08-31,400,229458322X-0,GAUGE PASSENGER SINGLE CAR AIR 90-130#,0.0,NaN,0.0,MPT,NaN,NaN,NaN,MPT,NaN
1,2021-08-31,400,2294586068-0,PISTON EMERGENCY F/26-F SERVICE,0.0,NaN,0.0,MPT,NaN,NaN,NaN,MPT,NaN
2,2021-08-31,402,2470300387-1,"ACTUATOR PBAM 8"" R/H F/900 KAWI SERIES WABCO T...",0.0,NaN,0.0,MPT,NaN,NaN,NaN,MPT,NaN
3,2021-08-31,402,2470300388-1,"ACTUATOR PBAM 8"" L/H F/900 KAWI SERIES WABCO ...",0.0,NaN,0.0,MPT,NaN,NaN,NaN,MPT,NaN
4,2021-08-31,400,2470300388-1,"ACTUATOR PBAM 8"" L/H F/900 KAWI SERIES WABCO ...",0.0,NaN,0.0,MPT,NaN,NaN,NaN,MPT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21256,2023-07-31,404,EMODKITROT-0,INTERIOR EMERGENCY DOOR RELEASE KIT FOR ROTEM,4,80.2771,321.11,NON-MPT,NaN,NaN,NaN,NON-MPT,NaN
21257,2023-07-31,402,ROTEM1YRAIR-0,1 YEAR ROTEM AIR PORTION LIST,2,183.246,366.49,NON-MPT,NaN,NaN,NaN,NON-MPT,NaN
21258,2023-07-31,400,ROTEM1YRAIR-0,1 YEAR ROTEM AIR PORTION LIST,3,183.246,549.74,NON-MPT,NaN,NaN,NaN,NON-MPT,NaN
21259,2023-07-31,402,ROTEMEMODKIT-0,ROTEM EXTERIOR EMOD KIT,7,50.3871,352.71,MPT,NaN,NaN,NaN,MPT,NaN


In [17]:
# Delete Queries (removing invalid transactions and parts)

# 2 or 97 IOH Suffixes (removing just the substring)
Initial_Inv_Updated['Part Suffix'] = Initial_Inv_Updated['Part Suffix'].str.replace('-98','')
Initial_Inv_Updated['Part Suffix'] = Initial_Inv_Updated['Part Suffix'].str.replace('-97','')
Initial_Inv_Updated['Part Suffix'] = Initial_Inv_Updated['Part Suffix'].str.replace('-2','')

# 2 or 97 or 98 (deleting the cell)
All_Txns_Updated['Last 2'] = All_Txns_Updated['Part Suffix'].str.strip().str[-2]
All_Txns_Updated['Last 3'] = All_Txns_Updated['Part Suffix'].str.strip().str[-3]

All_Txns_Updated.loc[All_Txns_Updated['Last 2'] == '-2', 'Part Suffix'] = ''
All_Txns_Updated.loc[All_Txns_Updated['Last 3'] == '-97', 'Part Suffix'] = ''
All_Txns_Updated.loc[All_Txns_Updated['Last 3'] == '-98', 'Part Suffix'] = ''

# Zero Value Txns (deleting the cell)
All_Txns_Updated = All_Txns_Updated.drop(All_Txns_Updated[All_Txns_Updated['TNX Value'] == '0'].index)

All_Txns_Updated = All_Txns_Updated.drop(columns=['Last 2', 'Last 3'])


In [18]:
# 3. Update matching Initial Values
# Carries forward parts on the previous month report that are no longer in the current month's inventory

# Inner Join Prior Month Tbl and Initial Inventory Report
Initial_Inv_Copy = copy.copy(Initial_Inv_Updated)
Initial_Inv_Copy = Initial_Inv_Copy.rename(columns={'Report Date': 'Future Report Date'})

Initial_Inv_Copy['Future Report Date'] = pd.to_datetime(Initial_Inv_Copy['Future Report Date'])

IIR_Prior_Merge = Prior_Month_Tbl.merge(Initial_Inv_Copy, how='inner', on=['Part Suffix', 'Location', 'Future Report Date'])


IIR_Prior_Merge = IIR_Prior_Merge.drop(columns=['Part Description_y', 'Initial MPT Status_y', 'Initial QOH_y', 'Initial Unit Price_y', 'Report Date'])

# renaming columns to keep only the left table's columns
IIR_Prior_Merge = IIR_Prior_Merge.rename(columns={'Part Description_x': 'Part Description', 'Initial MPT Status_x': 'Initial MPT Status', 'Initial QOH_x': 'Initial QOH', 'Initial Unit Price_x': 'Initial Unit Price',
                                                  'Future Report Date': 'Report Date'})

Initial_Inv_Updated['Report Date'] = pd.to_datetime(Initial_Inv_Updated['Report Date'])

Initial_Inv_Updated_Merged = pd.merge(Initial_Inv_Updated, IIR_Prior_Merge, on=['Report Date', 'Location', 'Part Suffix'],how='left')



In [19]:
# Replace null values
Initial_Inv_Updated_Merged['Initial MPT Status_x'] = Initial_Inv_Updated_Merged['Initial MPT Status_x'].fillna(Initial_Inv_Updated_Merged['Initial MPT Status_y'])

Initial_Inv_Updated_Merged['Initial QOH_x'] = Initial_Inv_Updated_Merged['Initial QOH_x'].fillna(Initial_Inv_Updated_Merged['Initial QOH_y'])

Initial_Inv_Updated_Merged['Initial Unit Price_x'] = Initial_Inv_Updated_Merged['Initial Unit Price_x'].fillna(Initial_Inv_Updated_Merged['Initial Unit Price_y'])

Initial_Inv_Updated_Merged['Initial Value OH_x'] = Initial_Inv_Updated_Merged['Initial Value OH_x'].fillna(Initial_Inv_Updated_Merged['Initial VOH'])


In [20]:
# drop columns from table on the right
Initial_Inv_Updated_Merged.drop(columns={'Part Description_y', 'Initial QOH_y', 'Initial Unit Price_y', 'Initial MPT Status_y', 'ID_y', 'Part Suffix Location_y', 'Part ID_y', 'Suffix_y', 'MPT Status_y',
                                'Final QOH_y', 'Final Unit Cost_y', 'Final Extended Cost_y', 'Initial Value OH_y', 'Last Issue Date_y', 'Initial VOH'}, axis=1, inplace=True)

# renaming columns to keep only the left table's columns
Initial_Inv_Updated_Merged = Initial_Inv_Updated_Merged.rename(columns={'ID_x' : 'ID', 'Part Suffix Location_x': 'Part Suffix Location', 'Part ID_x': 'Part ID', 'Suffix_x': 'Suffix',
                                                                        'Part Description_x': 'Part Description', 'MPT Status_x': 'MPT Status', 'Final QOH_x': 'Final QOH', 'Final Unit Cost_x': 'Final Unit Cost',
                                                                        'Final Extended Cost_x': 'Final Extended Cost', 'Initial MPT Status_x': 'Initial MPT Status', 'Initial QOH_x': 'Initial QOH',
                                                                        'Initial Unit Price_x': 'Initial Unit Price', 'Initial Value OH_x': 'Initial Value OH', 'Last Issue Date_x': 'Last Issue Date',
                                                                        })


In [21]:
# 4. Make Keolis IOH Table
# Final inventory with initial and final quantity on hand

Keolis_IOH_1 = Initial_Inv_Updated_Merged[['Report Date', 'Location', 'Part Suffix', 'Part Description', 
                                    'Initial QOH', 'Initial Unit Price', 'Initial Value OH', 'Initial MPT Status', 'Final QOH',
                                    'Final Unit Cost', 'Final Extended Cost', 'MPT Status', 'Last Issue Date']] # extract and order necessary columns

# renaming columns to match query
Keolis_IOH_1 = Keolis_IOH_1.rename(columns={'Part Suffix': 'Item Number-Sfx',
                                        'Part Description': 'Item Desc',
                                        'Final Unit Cost': 'Final Unit Price',
                                        'Final Extended Cost': 'Final VOH'})

# sort
Keolis_IOH_1.sort_values(['Report Date', 'Location', 'Item Number-Sfx'], ascending = [True, True, True], ignore_index=True)

Keolis_IOH_1['Report Date'] = pd.to_datetime(Keolis_IOH_1['Report Date']).dt.strftime('%Y-%m-%d') # 2023-02-28

In [22]:
Query_2_Unsorted = Query_2_Unsorted.rename(columns={'Future Report Date': 'Report Date', 'Initial VOH': 'Initial Value OH'})

Keolis_IOH = pd.concat([Keolis_IOH_1, Query_2_Unsorted], ignore_index=True)

Keolis_IOH['Report Date'] = pd.to_datetime(Keolis_IOH['Report Date']).dt.strftime('%Y-%m-%d') # 2023-02-28

Keolis_IOH

,Report Date,Location,Item Number-Sfx,Item Desc,Initial QOH,Initial Unit Price,Initial Value OH,Initial MPT Status,Final QOH,Final Unit Price,Final VOH,MPT Status,Last Issue Date
0,2021-11-30,400,1087474843-0,CONTACTOR 115 VAC 27 AMP 3 POLE,21.0,NaN,2957.43,MPT,21.0,NaN,2957.43,MPT,NaN
1,2021-11-30,400,1152256413-0,"RIVET COUNTERSUNK HD 3/16"" X 11/16; EMD 273621",40.0,NaN,28.0,MPT,40.0,NaN,28.0,MPT,NaN
2,2021-11-30,400,1171402395-0,SCREW; SCD; FLATHEAD; UNC-2A; F/KNOB(267140491...,6.0,NaN,8.82,MPT,6.0,NaN,8.82,MPT,NaN
3,2021-11-30,400,1241404022-0,SPRING COIL OUTER PER AAR STANDARD 12452A,67.0,NaN,5786.83,MPT,19.0,NaN,1641.04,MPT,NaN
4,2021-11-30,400,1241404023-0,SPRING COIL INNER PER AAR STANDARD,114.0,NaN,8329.05,MPT,66.0,NaN,4822.08,MPT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
439244,2023-07-31,400SW,2471405113-0,"IVNR CONTROLLER INSTALL KIT, GPS-WIFI, MBTA/PU...",32,17662,565184,MPT,NaN,NaN,NaN,MPT,NaN
439245,2023-07-31,405,0104500149-0,CLIP- RAIL; (25/BAG); PANDROL E-2055,320905,1.994,639884.57,NON-MPT,NaN,NaN,NaN,NON-MPT,NaN
439246,2023-07-31,405,0124014886-0,"PLATE- TIE- PANDROL- 6"" BASE FOR LAG SCREWS",41986,18.4228,773499.67,NON-MPT,NaN,NaN,NaN,NON-MPT,NaN
439247,2023-07-31,405,0599900207-0,TIE- CROSS- TREATED,15606,67.1969,1048674.82,NON-MPT,NaN,NaN,NaN,NON-MPT,NaN


In [23]:
Keolis_IOH.sort_values(['Report Date', 'Item Number-Sfx'], ascending = [True, True], ignore_index=True)

,Report Date,Location,Item Number-Sfx,Item Desc,Initial QOH,Initial Unit Price,Initial Value OH,Initial MPT Status,Final QOH,Final Unit Price,Final VOH,MPT Status,Last Issue Date
0,2021-07-31,400,000FILTE01-0,FILTER F/HOUSE MAINT 20 X 24 X 2,288.0,3.16,910.08,NON-MPT,288.0,3.16,910.08,NON-MPT,NaN
1,2021-07-31,400,000FILTE04-0,FILTER F/HOUSE MAINT 16 X 20 X 2,169.0,13.2372,2237.08,NON-MPT,169.0,13.2372,2237.08,NON-MPT,NaN
2,2021-07-31,400,000FILTE07-0,FILTER F/HOUSE MAINT 16 X 25 X 2,66.0,2.9879,197.2,NON-MPT,66.0,2.9879,197.2,NON-MPT,NaN
3,2021-07-31,400,000FILTE08-0,FILTER F/HOUSE MAINT 16 X 16 X 2,72.0,5.87,422.64,NON-MPT,72.0,5.87,422.64,NON-MPT,NaN
4,2021-07-31,400,000FILTE09-0,FILTER F/HOUSE MAINT 13-1/2 X 15-1/2 X 2,72.0,10.69,769.68,NON-MPT,72.0,10.69,769.68,NON-MPT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
439244,2023-07-31,404,EMODKITROT-0,INTERIOR EMERGENCY DOOR RELEASE KIT FOR ROTEM,4,80.2771,321.11,NON-MPT,NaN,NaN,NaN,NON-MPT,NaN
439245,2023-07-31,402,ROTEM1YRAIR-0,1 YEAR ROTEM AIR PORTION LIST,2,183.246,366.49,NON-MPT,NaN,NaN,NaN,NON-MPT,NaN
439246,2023-07-31,400,ROTEM1YRAIR-0,1 YEAR ROTEM AIR PORTION LIST,3,183.246,549.74,NON-MPT,NaN,NaN,NaN,NON-MPT,NaN
439247,2023-07-31,402,ROTEMEMODKIT-0,ROTEM EXTERIOR EMOD KIT,7,50.3871,352.71,MPT,NaN,NaN,NaN,MPT,NaN


In [24]:
# 5. Make Final Keolis IOH Table
Initial_Final_End = Keolis_IOH

Initial_Final_End

,Report Date,Location,Item Number-Sfx,Item Desc,Initial QOH,Initial Unit Price,Initial Value OH,Initial MPT Status,Final QOH,Final Unit Price,Final VOH,MPT Status,Last Issue Date
0,2021-11-30,400,1087474843-0,CONTACTOR 115 VAC 27 AMP 3 POLE,21.0,NaN,2957.43,MPT,21.0,NaN,2957.43,MPT,NaN
1,2021-11-30,400,1152256413-0,"RIVET COUNTERSUNK HD 3/16"" X 11/16; EMD 273621",40.0,NaN,28.0,MPT,40.0,NaN,28.0,MPT,NaN
2,2021-11-30,400,1171402395-0,SCREW; SCD; FLATHEAD; UNC-2A; F/KNOB(267140491...,6.0,NaN,8.82,MPT,6.0,NaN,8.82,MPT,NaN
3,2021-11-30,400,1241404022-0,SPRING COIL OUTER PER AAR STANDARD 12452A,67.0,NaN,5786.83,MPT,19.0,NaN,1641.04,MPT,NaN
4,2021-11-30,400,1241404023-0,SPRING COIL INNER PER AAR STANDARD,114.0,NaN,8329.05,MPT,66.0,NaN,4822.08,MPT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
439244,2023-07-31,400SW,2471405113-0,"IVNR CONTROLLER INSTALL KIT, GPS-WIFI, MBTA/PU...",32,17662,565184,MPT,NaN,NaN,NaN,MPT,NaN
439245,2023-07-31,405,0104500149-0,CLIP- RAIL; (25/BAG); PANDROL E-2055,320905,1.994,639884.57,NON-MPT,NaN,NaN,NaN,NON-MPT,NaN
439246,2023-07-31,405,0124014886-0,"PLATE- TIE- PANDROL- 6"" BASE FOR LAG SCREWS",41986,18.4228,773499.67,NON-MPT,NaN,NaN,NaN,NON-MPT,NaN
439247,2023-07-31,405,0599900207-0,TIE- CROSS- TREATED,15606,67.1969,1048674.82,NON-MPT,NaN,NaN,NaN,NON-MPT,NaN


In [25]:
# Export and overwrite original historical files

Initial_Final_End.to_excel(Initial_Final_Path)

All_Txns_Updated.to_excel(All_Txns_Path)

Initial_Inv_Updated.to_excel(Initial_Inv_Path)

In [26]:
# The above files are used in the Tableau Prep flow

In [27]:
# %%
# Copy dataframes to historical backup folder (in case a report needs to be reran)
import shutil

# pulls date of report to add to file name
report_date = datetime.now() - pd.offsets.MonthEnd(n=1)
year_month = str(report_date.year) + '-' + str('%02d' % report_date.month) # format: '2023-02'

# Initial Final
src_path_IF = Initial_Final_Path # overwritten historical path
dst_path_IF = 'O:\\PP-PROLOG\\VENDOR_MANAGEMENT\\Vendor Management Team\\Co-Ops\\BridgetLeary\\Keolis\\New Keolis\\Historical Data\\Initial and Final Inventory\\Initial_Final_' + year_month + '.xlsx'
shutil.copy(src_path_IF, dst_path_IF)

# All Txns
src_path_AT = All_Txns_Path # overwritten historical path
dst_path_AT = 'O:\\PP-PROLOG\\VENDOR_MANAGEMENT\\Vendor Management Team\\Co-Ops\\BridgetLeary\\Keolis\\New Keolis\\Historical Data\\All Transactions\\All_Txns_' + year_month + '.xlsx'
shutil.copy(src_path_AT, dst_path_AT)

# Initial Inv
src_path_II = Initial_Inv_Path # overwritten historical path
dst_path_II = 'O:\\PP-PROLOG\\VENDOR_MANAGEMENT\\Vendor Management Team\\Co-Ops\\BridgetLeary\\Keolis\\New Keolis\\Historical Data\\Initial Inventory\\Initial_Inv_' + year_month + '.xlsx'
shutil.copy(src_path_II, dst_path_II)


'O:\\PP-PROLOG\\VENDOR_MANAGEMENT\\Vendor Management Team\\Co-Ops\\BridgetLeary\\Keolis\\New Keolis\\Historical Data\\Initial Inventory\\Initial_Inv_2023-06.xlsx'